In [40]:
spark.stop()

In [41]:
import os
import sys
os.environ["PYSPARK_PYTHON"]='/opt/anaconda/envs/bd9/bin/python'
os.environ["SPARK_HOME"]='/usr/hdp/current/spark2-client'
os.environ["PYSPARK_SUBMIT_ARGS"]='--num-executors 2 pyspark-shell'

spark_home = os.environ.get('SPARK_HOME', None)
if not spark_home:
    raise ValueError('SPARK_HOME environment variable is not set')

sys.path.insert(0, os.path.join(spark_home, 'python'))
sys.path.insert(0, os.path.join(spark_home, 'python/lib/py4j-0.10.7-src.zip'))
# exec(open(os.path.join(spark_home, 'python/pyspark/shell.py')).read())

In [42]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark import Row
import json

conf = SparkConf()

spark = (SparkSession
         .builder
         .config(conf=conf)
         .appName("zhukov_lab1")
         .getOrCreate())

In [43]:
spark

In [50]:
sc = spark.sparkContext

In [51]:
rdd = sc.textFile("/labs/laba01/ml-100k/u.data")

In [52]:
rdd.take(5)

['196\t242\t3\t881250949',
 '186\t302\t3\t891717742',
 '22\t377\t1\t878887116',
 '244\t51\t2\t880606923',
 '166\t346\t1\t886397596']

In [53]:
rdd.getNumPartitions()

2

In [54]:
TARGET_ID = "22"

In [55]:
def split_films(line):
    """:return: tuple (rating, item_id)"""
    splitted_line = line.split("\t")
    return splitted_line[2], splitted_line[1]

In [56]:
film_rating_rdd = rdd.map(split_films)

In [57]:
film_rating_rdd.take(5)

[('3', '242'), ('3', '302'), ('1', '377'), ('2', '51'), ('1', '346')]

In [58]:
target_film = film_rating_rdd.filter(lambda x: x[1]==TARGET_ID).countByKey().items()

In [59]:
all_films = film_rating_rdd.countByKey().items()

In [60]:
json_data = {}
json_data["hist_film"] = [x[1] for x in sorted(target_film, key=lambda x: x[0])]
json_data["hist_all"] = [x[1] for x in sorted(all_films, key=lambda x: x[0])]

In [61]:
json_data

{'hist_film': [5, 14, 46, 98, 134],
 'hist_all': [6110, 11370, 27145, 34174, 21201]}

In [62]:
with open("lab01.json", "w", encoding="utf-8") as json_file:
    json_file.write(json.dumps(json_data, indent=4))

In [63]:
with open("lab01.json", "r", encoding="utf-8") as json_file:
    result = json.load(json_file)

In [64]:
print(result)

{'hist_film': [5, 14, 46, 98, 134], 'hist_all': [6110, 11370, 27145, 34174, 21201]}


In [66]:
# !hdfs dfs -put lab01.json /user/alexander.zhukov/

In [65]:
sc.stop()